In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import re

import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [2]:
# CC - Coordinating Conjunction
# CD - Cardinal Number
# DT - Determiner
# EX - Existential There
# FW - Foreign Word
# IN - Preposition or Subordinating Conjunction
# JJ - Adjective
# JJR - Adjective, Comparative
# JJS - Adjective, Superlative
# LS - List Item Marker
# MD - Modal
# NN - Noun, Singular or Mass
# NNS - Noun, Plural
# NNP - Proper Noun, Singular
# NNPS - Proper Noun, Plural
# PDT - Predeterminer
# POS - Possessive Ending
# PRP - Personal Pronoun
# PRP$ - Possessive Pronoun
# RB - Adverb
# RBR - Adverb, Comparative
# RBS - Adverb, Superlative
# RP - Particle
# SYM - Symbol
# TO - to
# UH - Interjection
# VB - Verb, Base Form
# VBD - Verb, Past Tense
# VBG - Verb, Gerund or Present Participle
# VBN - Verb, Past Participle
# VBP - Verb, Non-3rd Person Singular Present
# VBZ - Verb, 3rd Person Singular Present
# WDT - Wh-Determiner
# WP - Wh-Pronoun
# WP$ - Possessive Wh-Pronoun
# WRB - Wh-Adverb

In [3]:
statement_df = pd.read_csv("Bank_Statement.csv")
def split_string(Split_sting):
    new = Split_sting.split(" ")
    return new
def remove_special_characters(string):
    pattern = r'[~&]'
    cleaned_string = re.sub(pattern, ' ', string)
    return cleaned_string

In [4]:
from collections import defaultdict
def link_liked_strings(string):
    tokens = nltk.word_tokenize(string, language="english")
    pos_tags = nltk.pos_tag(tokens)
    pos_dict = {word: pos for word, pos in pos_tags}

    grouped_keys_dict = defaultdict(list)

    for key, value in pos_dict.items():
        grouped_keys_dict[value].append(key)


    result_list = [' '.join(keys) for keys in grouped_keys_dict.values()][0]
    return result_list

In [5]:
#Remove visa and on
elements_to_remove = ["VISA", "ON"]
def remove_visa_on(string):
    String_list = split_string(string)
    filtered_list = [element for element in String_list if element not in elements_to_remove] 
    new = " ".join(filtered_list)
    return new

In [6]:
statement_df["Narrative"] = statement_df["Narrative"].apply(lambda x: remove_special_characters(x))
statement_df["Narrative"] = statement_df["Narrative"].apply(lambda x: link_liked_strings(x))
statement_df["Narrative"] = statement_df["Narrative"].apply(lambda x: remove_visa_on(x))

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
import joblib
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# nltk.download('stopwords')
train_data, test_data = train_test_split(statement_df, test_size=0.2, random_state=42)
# def preprocess_text(text):
#     stop_words = set(stopwords.words('english'))
#     ps = PorterStemmer()
#     cleaned_text = ' '.join([ps.stem(word) for word in text.split() if word.lower() not in stop_words])
#     return cleaned_text

# Initialize the pipeline with custom preprocessing and TfidfVectorizer
# model = make_pipeline(
#     TfidfVectorizer(preprocessor=preprocess_text),
#     MultinomialNB()
# )
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

model.fit(train_data['Narrative'], train_data['Label'])

predictions = model.predict(test_data['Narrative'])

accuracy = accuracy_score(test_data['Label'], predictions)
classification_rep = classification_report(test_data['Label'], predictions)


# Assuming 'model' is the trained model
joblib.dump(model, 'your_model_filename.joblib')
print(f"Model Accuracy: {accuracy}")
print("\nClassification Report:")
print(classification_rep)

Model Accuracy: 0.703125

Classification Report:
               precision    recall  f1-score   support

        Booze       0.84      0.80      0.82        20
Entertainment       0.00      0.00      0.00         3
         Food       0.00      0.00      0.00         7
      Receive       1.00      0.67      0.80         3
 Subscription       0.00      0.00      0.00         1
     Supplies       0.00      0.00      0.00         3
     Transfer       0.83      1.00      0.91         5
       Travel       0.63      1.00      0.77        22

     accuracy                           0.70        64
    macro avg       0.41      0.43      0.41        64
 weighted avg       0.59      0.70      0.63        64



C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\james\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344